In [583]:
import pandas as pd
df1 = pd.read_csv('edges3.csv')
df2 = pd.read_excel('node3_cml.xlsx')

In [584]:
df1

,l_id,v_id_1,v_id_2,voltage,cables,wires,length_m,r,x,c,i_th_max
0,1.0,1.0,2.0,220000.0,3.0,2.0,43379.0,1.735160,13.881280,498.8585,112.78540
1,2.0,3.0,4.0,380000.0,6.0,4.0,72686.0,0.227144,18.171500,995.7982,1511.86880
2,3.0,5.0,6.0,220000.0,6.0,2.0,33943.0,0.678860,10.861760,390.3445,176.50360
3,4.0,7.0,5.0,380000.0,3.0,4.0,33471.0,0.209194,8.367750,458.5527,348.09840
4,5.0,8.0,9.0,380000.0,6.0,4.0,28163.0,0.088009,7.040750,385.8331,585.79040
...,...,...,...,...,...,...,...,...,...,...,...
760,761.0,474.0,476.0,220000.0,9.0,1.8,770.0,0.038376,0.253184,9.0988,2.10912
761,762.0,475.0,476.0,220000.0,9.0,2.4,661.0,0.020584,0.196608,7.0656,2.11640
762,763.0,477.0,479.0,220000.0,12.0,2.0,3315.0,0.126104,1.008832,36.2549,8.19676
763,764.0,225.0,479.0,220000.0,12.0,2.0,897.0,0.033360,0.289088,10.3891,2.70972


In [585]:
df2

,v_id,lon,lat,typ,voltage,eigen_centrality,closeness_centrality,clustering_coefficient,load_centrality,avg_shortest_path_length,average_neighbor_degree,node_strength,criticality_label
0,1,9.522576,52.360409,substation,220000,9.305167e-51,0.000000,0.00,0.000000,2.222222,3.000000,220000,Low
1,2,9.113210,52.543853,substation,220000,1.838136e-31,0.006834,0.05,0.000197,1.500000,1.333333,1260000,Severe
2,3,9.389745,52.026313,substation,380000,7.071928e-44,0.004184,0.05,0.000171,3.000000,1.000000,1900000,Severe
3,4,9.125265,52.538264,substation,380000,1.813950e-37,0.006538,0.00,0.000395,2.350000,2.000000,1900000,Severe
4,5,10.366275,52.284647,substation,380000,7.071928e-44,0.004184,0.00,0.000110,1.500000,1.333333,2400000,Severe
...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,475,8.872260,50.129725,substation,110000,9.305167e-51,0.000000,0.00,0.000000,0.500000,0.000000,220000,Low
475,476,8.743261,50.162338,auxillary_T_node,220000,6.141411e-44,0.006276,0.00,0.000000,0.000000,0.000000,660000,High
476,477,7.957537,47.572159,substation,220000,9.305167e-51,0.000000,0.00,0.000000,0.500000,0.000000,220000,Low
477,478,7.958491,47.579148,generator,380000,9.305167e-51,0.000000,0.00,0.000000,0.500000,0.000000,220000,Low


In [586]:
#Handle Categorical Variable in df2
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# One-hot encode the 'typ' column
encoder = OneHotEncoder(sparse=False)
typ_encoded = encoder.fit_transform(df2[['typ']])

# Convert to DataFrame and concatenate with df2
typ_encoded_df = pd.DataFrame(typ_encoded, columns=encoder.get_feature_names_out(['typ']))
df2 = pd.concat([df2.drop('typ', axis=1), typ_encoded_df], axis=1)

/Applications/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [587]:
#Normalize Numerical Features in df2 and df1
from sklearn.preprocessing import StandardScaler

# Standardize node features in df2
node_feature_cols = [col for col in df2.columns if col not in ['v_id', 'criticality_label', *encoder.get_feature_names_out(['typ'])]]
scaler = StandardScaler()
df2[node_feature_cols] = scaler.fit_transform(df2[node_feature_cols])

# Standardize edge features in df1
edge_feature_cols = ['voltage', 'cables', 'wires', 'length_m', 'r', 'x', 'c', 'i_th_max']
df1[edge_feature_cols] = scaler.fit_transform(df1[edge_feature_cols])

In [588]:
#Convert Data for PyTorch Geometric
#import torch
#from torch_geometric.data import Data
#from sklearn.preprocessing import LabelEncoder

# Assuming df1 and df2 are already defined and 'edge_feature_cols' is defined
# Example: edge_feature_cols = ['voltage', 'cables', 'wires', 'length_m', 'r', 'x', 'c', 'i_th_max']

# Prepare node features
#data_x = torch.tensor(df2.drop(['v_id', 'criticality_label'], axis=1).values, dtype=torch.float)

# Prepare edge index
#edge_index = torch.tensor(df1[['v_id_1', 'v_id_2']].values, dtype=torch.long).t().contiguous()

# Prepare edge features
#data_edge_attr = torch.tensor(df1[edge_feature_cols].values, dtype=torch.float)

# Encode target labels
#label_encoder = LabelEncoder()
#df2['criticality_label_encoded'] = label_encoder.fit_transform(df2['criticality_label'])
#data_y = torch.tensor(df2['criticality_label_encoded'].values, dtype=torch.long)

# Create Data object
#data1 = Data(x=data_x, edge_index=edge_index, edge_attr=data_edge_attr, y=data_y)
import torch
from torch_geometric.data import Data
from sklearn.preprocessing import LabelEncoder

# Assuming df1 and df2 are already defined and 'edge_feature_cols' is defined
# Example: edge_feature_cols = ['voltage', 'cables', 'wires', 'length_m', 'r', 'x', 'c', 'i_th_max']

# Prepare node features
data_x = torch.tensor(df2.drop(['v_id', 'criticality_label'], axis=1).values, dtype=torch.float)

# Prepare edge index
# Subtract 1 from each node index to convert to 0-based indexing
edge_index = torch.tensor(df1[['v_id_1', 'v_id_2']].values, dtype=torch.long) - 1
edge_index = edge_index.t().contiguous()

# Prepare edge features
data_edge_attr = torch.tensor(df1[edge_feature_cols].values, dtype=torch.float)

# Encode target labels
label_encoder = LabelEncoder()
df2['criticality_label_encoded'] = label_encoder.fit_transform(df2['criticality_label'])
data_y = torch.tensor(df2['criticality_label_encoded'].values, dtype=torch.long)

# Create Data object
data1 = Data(x=data_x, edge_index=edge_index, edge_attr=data_edge_attr, y=data_y)

In [589]:
print(data1)

Data(x=[479, 14], edge_index=[2, 765], edge_attr=[765, 8], y=[479])


In [590]:
import torch
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.utils import to_dense_adj

# Assuming 'data1' is already defined as per your previous code

def accuracy(y_pred, y_true):
    return torch.sum(y_pred == y_true) / len(y_true)

class VanillaGNNLayer(torch.nn.Module):
    def __init__(self, dim_in, dim_out):
        super().__init__()
        self.linear = Linear(dim_in, dim_out, bias=False)

    def forward(self, x, adjacency):
        x = self.linear(x)
        x = torch.sparse.mm(adjacency, x)  # Multiplication with A
        return x

# Convert the edge index to adjacency matrix and add self-loops
adjacency = to_dense_adj(data1.edge_index)[0]
adjacency += torch.eye(len(adjacency))
num_nodes = data1.num_nodes
indices = torch.randperm(num_nodes)

# Split indices for training, validation, and testing
train_size = int(num_nodes * 0.8)
val_size = int(num_nodes * 0.1)
test_size = num_nodes - (train_size + val_size)

train_mask = torch.zeros(num_nodes, dtype=torch.bool)
val_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)

train_mask[indices[:train_size]] = True
val_mask[indices[train_size:train_size + val_size]] = True
test_mask[indices[train_size + val_size:]] = True

# Assign these masks to your data object
data1.train_mask = train_mask
data1.val_mask = val_mask
data1.test_mask = test_mask
class VanillaGNN(torch.nn.Module):
    def __init__(self, dim_in, dim_h, dim_out):
        super().__init__()
        self.gnn1 = VanillaGNNLayer(dim_in, dim_h)
        self.gnn2 = VanillaGNNLayer(dim_h, dim_out)

    def forward(self, x, adjacency):
        h = self.gnn1(x, adjacency)
        h = torch.relu(h)
        h = self.gnn2(h, adjacency)
        return F.log_softmax(h, dim=1)

    def fit(self, data, epochs, adjacency):
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.parameters(), lr=0.01, weight_decay=5e-4)
        self.train()
        for epoch in range(epochs+1):
            optimizer.zero_grad()
            out = self(data.x, adjacency)
            loss = criterion(out[data.train_mask], data.y[data.train_mask])
            acc = accuracy(out[data.train_mask].argmax(dim=1), data.y[data.train_mask])
            loss.backward()
            optimizer.step()
            if epoch % 20 == 0:
                val_loss = criterion(out[data.val_mask], data.y[data.val_mask])
                val_acc = accuracy(out[data.val_mask].argmax(dim=1), data.y[data.val_mask])
                print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Train Acc: {acc*100:>5.2f}% | Val Loss: {val_loss:.2f} | Val Acc: {val_acc*100:.2f}%')

    def test(self, data, adjacency):
        self.eval()
        out = self(data.x, adjacency)
        acc = accuracy(out.argmax(dim=1)[data.test_mask], data.y[data.test_mask])
        return acc

# Initialize the GNN
num_features = data1.num_features
num_classes = int(data1.y.max()) + 1
gnn = VanillaGNN(num_features, 16, num_classes)
print(gnn)

# Make sure to add train_mask, val_mask, and test_mask to data1 as needed

# Fit and test the model
gnn.fit(data1, epochs=100, adjacency=adjacency)
acc = gnn.test(data1, adjacency)
print(f'\nGNN test accuracy: {acc*100:.2f}%')

VanillaGNN(
  (gnn1): VanillaGNNLayer(
    (linear): Linear(in_features=14, out_features=16, bias=False)
  )
  (gnn2): VanillaGNNLayer(
    (linear): Linear(in_features=16, out_features=4, bias=False)
  )
)
Epoch   0 | Train Loss: 1.890 | Train Acc: 22.72% | Val Loss: 1.81 | Val Acc: 21.28%
Epoch  20 | Train Loss: 1.241 | Train Acc: 40.99% | Val Loss: 1.37 | Val Acc: 38.30%
Epoch  40 | Train Loss: 1.151 | Train Acc: 50.39% | Val Loss: 1.28 | Val Acc: 46.81%
Epoch  60 | Train Loss: 1.081 | Train Acc: 53.79% | Val Loss: 1.29 | Val Acc: 48.94%
Epoch  80 | Train Loss: 1.031 | Train Acc: 56.14% | Val Loss: 1.31 | Val Acc: 48.94%
Epoch 100 | Train Loss: 0.992 | Train Acc: 56.92% | Val Loss: 1.40 | Val Acc: 40.43%

GNN test accuracy: 38.78%


In [579]:
def train_and_evaluate_gnn(learning_rate, hidden_units, epochs, adjacency, data, train_mask, val_mask, test_mask):
    # Adjust the model's parameters
    gnn.gnn1.linear = Linear(data.num_features, hidden_units, bias=False)
    gnn.gnn2.linear = Linear(hidden_units, int(data.y.max()) + 1, bias=False)

    # Define the optimizer with the given learning rate
    optimizer = torch.optim.Adam(gnn.parameters(), lr=learning_rate, weight_decay=5e-4)

    # Training loop
    criterion = torch.nn.CrossEntropyLoss()
    gnn.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = gnn(data.x, adjacency)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

    # Evaluate on test set
    gnn.eval()
    out = gnn(data.x, adjacency)
    acc = accuracy(out.argmax(dim=1)[test_mask], data.y[test_mask])
    return acc

# Grid search parameters
learning_rates = [0.01, 0.005, 0.001]
hidden_units_options = [16, 32, 64]
epochs = 100

# Variables to store the best model's performance
best_accuracy = 0.0
best_lr = 0
best_hu = 0

for lr in learning_rates:
    for hu in hidden_units_options:
        acc = train_and_evaluate_gnn(lr, hu, epochs, adjacency, data1, data1.train_mask, data1.val_mask, data1.test_mask)
        print(f"Learning Rate: {lr}, Hidden Units: {hu}, Test Accuracy: {acc:.4f}")

        if acc > best_accuracy:
            best_accuracy = acc
            best_lr = lr
            best_hu = hu

print(f"\nBest Model - Learning Rate: {best_lr}, Hidden Units: {best_hu}, Test Accuracy: {best_accuracy:.4f}")

Learning Rate: 0.01, Hidden Units: 16, Test Accuracy: 0.4898
Learning Rate: 0.01, Hidden Units: 32, Test Accuracy: 0.4694
Learning Rate: 0.01, Hidden Units: 64, Test Accuracy: 0.4286
Learning Rate: 0.005, Hidden Units: 16, Test Accuracy: 0.4286
Learning Rate: 0.005, Hidden Units: 32, Test Accuracy: 0.4898
Learning Rate: 0.005, Hidden Units: 64, Test Accuracy: 0.4898
Learning Rate: 0.001, Hidden Units: 16, Test Accuracy: 0.4490
Learning Rate: 0.001, Hidden Units: 32, Test Accuracy: 0.4694
Learning Rate: 0.001, Hidden Units: 64, Test Accuracy: 0.5102

Best Model - Learning Rate: 0.001, Hidden Units: 64, Test Accuracy: 0.5102


In [580]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

# Assuming 'data1' is already defined with x, edge_index, and y

# Create train, validation, and test masks
num_nodes = data1.num_nodes
indices = torch.randperm(num_nodes)

train_size = int(num_nodes * 0.8)
val_size = int(num_nodes * 0.1)

train_mask = torch.zeros(num_nodes, dtype=torch.bool)
val_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)

train_mask[indices[:train_size]] = True
val_mask[indices[train_size:train_size + val_size]] = True
test_mask[indices[train_size + val_size:]] = True

data1.train_mask = train_mask
data1.val_mask = val_mask
data1.test_mask = test_mask

# Define accuracy function
def accuracy(pred_y, y):
    return ((pred_y == y).sum() / len(y)).item()

# Define GCN model
class GCN(torch.nn.Module):
    """Graph Convolutional Network"""
    def __init__(self, dim_in, dim_h, dim_out):
        super().__init__()
        self.gcn1 = GCNConv(dim_in, dim_h)
        self.gcn2 = GCNConv(dim_h, dim_out)

    def forward(self, x, edge_index):
        h = self.gcn1(x, edge_index)
        h = torch.relu(h)
        h = self.gcn2(h, edge_index)
        return F.log_softmax(h, dim=1)

    def fit(self, data, epochs):
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.parameters(), lr=0.01, weight_decay=5e-4)
        self.train()
        for epoch in range(epochs+1):
            optimizer.zero_grad()
            out = self(data.x, data.edge_index)
            loss = criterion(out[data.train_mask], data.y[data.train_mask])
            acc = accuracy(out[data.train_mask].argmax(dim=1), data.y[data.train_mask])
            loss.backward()
            optimizer.step()
            if(epoch % 20 == 0):
                val_loss = criterion(out[data.val_mask], data.y[data.val_mask])
                val_acc = accuracy(out[data.val_mask].argmax(dim=1), data.y[data.val_mask])
                print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Train Acc: {acc*100:>5.2f}% | Val Loss: {val_loss:.2f} | Val Acc: {val_acc*100:.2f}%')

    @torch.no_grad()
    def test(self, data):
        self.eval()
        out = self(data.x, data.edge_index)
        predictions = out.argmax(dim=1)
        acc = accuracy(predictions[data.test_mask], data.y[data.test_mask])
        return acc, predictions

# Initialize and train the GCN model
num_features = data1.num_features
num_classes = int(data1.y.max()) + 1
gcn = GCN(num_features, 16, num_classes)
print(gcn)
gcn.fit(data1, epochs=100)

# Test the model
acc, pred = gcn.test(data1)
print(f'GCN test accuracy: {acc*100:.2f}%')
#print(f'GCN predictions: {pred}')

GCN(
  (gcn1): GCNConv(14, 16)
  (gcn2): GCNConv(16, 4)
)
Epoch   0 | Train Loss: 1.958 | Train Acc: 22.19% | Val Loss: 2.12 | Val Acc: 27.66%
Epoch  20 | Train Loss: 1.235 | Train Acc: 45.43% | Val Loss: 1.32 | Val Acc: 38.30%
Epoch  40 | Train Loss: 1.149 | Train Acc: 47.52% | Val Loss: 1.23 | Val Acc: 44.68%
Epoch  60 | Train Loss: 1.097 | Train Acc: 52.22% | Val Loss: 1.25 | Val Acc: 40.43%
Epoch  80 | Train Loss: 1.058 | Train Acc: 55.09% | Val Loss: 1.25 | Val Acc: 38.30%
Epoch 100 | Train Loss: 1.020 | Train Acc: 56.92% | Val Loss: 1.26 | Val Acc: 36.17%
GCN test accuracy: 38.78%


In [581]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

# Assuming 'data1' is already defined with x, edge_index, and y

# Create train, validation, and test masks
num_nodes = data1.num_nodes
indices = torch.randperm(num_nodes)

train_size = int(num_nodes * 0.8)
val_size = int(num_nodes * 0.1)

train_mask = torch.zeros(num_nodes, dtype=torch.bool)
val_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)

train_mask[indices[:train_size]] = True
val_mask[indices[train_size:train_size + val_size]] = True
test_mask[indices[train_size + val_size:]] = True

data1.train_mask = train_mask
data1.val_mask = val_mask
data1.test_mask = test_mask

def accuracy(y_pred, y_true):
    return torch.sum(y_pred == y_true) / len(y_true)
class GAT(torch.nn.Module):
    def __init__(self, dim_in, dim_h, dim_out, heads=8):
        super().__init__()
        self.gat1 = GATv2Conv(dim_in, dim_h, heads=heads)
        self.gat2 = GATv2Conv(dim_h*heads, dim_out,
heads=1)
    def forward(self, x, edge_index):
        h = F.dropout(x, p=0.6, training=self.training)
        h = self.gat1(h, edge_index)
        h = F.elu(h)
        h = F.dropout(h, p=0.6, training=self.training)
        h = self.gat2(h, edge_index)
        return F.log_softmax(h, dim=1)
    def fit(self, data, epochs):
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.parameters(),
lr=0.01, weight_decay=0.01)
        self.train()
        for epoch in range(epochs+1):
            optimizer.zero_grad()
            out = self(data.x, data.edge_index)
            loss = criterion(out[data.train_mask],
data.y[data.train_mask])
            acc = accuracy(out[data.train_mask].argmax(dim=1), data.y[data.train_mask])
            loss.backward()
            optimizer.step()
            if(epoch % 20 == 0):
                val_loss = criterion(out[data.val_mask],
data.y[data.val_mask])
                val_acc = accuracy(out[data.val_mask].
argmax(dim=1), data.y[data.val_mask])
                print(f'Epoch {epoch:>3} | Train Loss:{loss:.3f} | Train Acc: {acc*100:>5.2f}% | Val Loss:{val_loss:.2f} | Val Acc: {val_acc*100:.2f}%')
    @torch.no_grad()
    def test(self, data):
        self.eval()
        out = self(data.x, data.edge_index)
        acc = accuracy(out.argmax(dim=1)[data.test_mask],
    data.y[data.test_mask])
        return acc
    
num_features = data1.num_features
num_classes = int(data1.y.max()) + 1
gat = GAT(num_features, 32, num_classes)
gat.fit(data, epochs=100)

acc = gat.test(data)
print(f'GAT test accuracy: {acc*100:.2f}%')

NameError: name 'GATv2Conv' is not defined